Player/match dataset creation

For each serie A (already happened) match and player, a database entry is generated, with this form:
    
[y = player votes data, player data, team data, vs team data, opp team data, vs opp team data]

In [ ]:
import pandas as pd

Load team data from FBref

In [ ]:
rcsv = pd.read_csv('fbref_data/teams.csv')   
team = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/teams_vs.csv')   
vs_team = pd.DataFrame(rcsv)

for i in range(1, team.shape[1]):
    team = team.rename(columns = {team.columns[i] : 'team_' + team.columns[i]})
    
for i in range(1, vs_team.shape[1]):
    vs_team = vs_team.rename(columns = {vs_team.columns[i] : 'vs_team_' + vs_team.columns[i]})
    
vs_team.pop('team')

team_data = pd.concat([team, vs_team], axis = 1)

team_data['team_idx'] = team_data['team']
for i in range(team_data.shape[0]):
    team_data.loc[i, 'team_idx'] = team_data.loc[i, 'team_idx'].split(' ')[-1]

team_data = team_data.set_index('team_idx')

team_data



In [ ]:
team_data.to_excel('mid_outputs/team_data.xlsx')

Load data generated from players_dataset_creation

In [ ]:
rx = pd.read_excel('mid_outputs/players_stats.xlsx', index_col = 3)   
players = pd.DataFrame(rx)

players


Function for merging match data (player, team, opponent team)

In [ ]:
def player_match_data(player, pteam, oppteam):
    if(not(player in players.index)):
        return None
    
    pdata = players.loc[[player]]
    
    pteam_stats = team_data.loc[[pteam]].rename(index = {pteam : player})
    
    oppteam_stats = team_data.loc[[oppteam]].rename(index = {oppteam : player})
    
    oppteam_stats = oppteam_stats.rename(lambda x: 'opp_' + x, axis='columns')
    
    out = pd.concat([pdata, pteam_stats, oppteam_stats], axis = 1)
    
    return(out)
    

Output to excel file data for a player. 

The file is copied and there the desired features are manually selected and printed.

In [ ]:
test = player_match_data('Barella', 'Inter', 'Atalanta')
test.to_excel('tmp/playermatchdata.xlsx')

test

In [ ]:
pteam = 'Inter'
pteam_stats = team_data.loc[[pteam]]
pteam_stats.index[0]

In [ ]:
rx = pd.read_excel('tmp/playermatchdata_sel.xlsx', index_col = 0, header = None)   

out = list()

for i in range(1, rx.shape[1]):
        if(rx[i][2] == "x"):
            out.append(rx[i][0])
            
out

Define and select the dataset features: abs features and rel features.
    
Rel features are divided by the player minutes played.

Rel features (game corrected) are divided by the player number of games instead of minutes. 
For example, a player who plays 20 minutes and scores, would be to match for stats averaging. It is better to consider the goals made per match.

In [ ]:
features_abs = ['r',
 'games',
 'games_starts',                
 'minutes',
 'shots_on_target_pct',
 'goals_per_shot',
 'goals_per_shot_on_target',
 'passes_pct',
 #'dribble_tackles_pct',
 #'dribbles_completed_pct',
 'aerials_won_pct',
 'team_possession',
 'team_goals_assists_per90',
 'team_goals_pens_per90',
 'team_goals_assists_pens_per90',
 'team_xg_per90',
 'team_gk_goals_against_per90',
 'team_gk_save_pct',
 'team_gk_clean_sheets_pct',
 'team_passes_pct',
 'team_passes_pct_medium',
 'team_passes_pct_long',
 'team_sca_per90',
 'team_gca_per90',
 #'team_dribble_tackles_pct',
 'team_aerials_won_pct',
 'vs_team_possession',
 'vs_team_goals_per90',
 'vs_team_assists_per90',
 'vs_team_xg_per90',
 'vs_team_gk_save_pct',
 'vs_team_gk_clean_sheets_pct',
 'vs_team_gk_pct_passes_launched',
 'vs_team_gk_crosses_stopped_pct',
 'vs_team_shots_on_target_per90',
 'vs_team_passes_pct',
 'vs_team_passes_pct_short',
 'vs_team_passes_pct_medium',
 'vs_team_passes_pct_long',
 'vs_team_sca_per90',
 'vs_team_gca_per90',
 #'vs_team_dribble_tackles_pct',
 #'vs_team_dribbles_completed_pct',
 'vs_team_aerials_won_pct',
 'opp_team_possession',
 'opp_team_goals_assists_per90',
 'opp_team_goals_pens_per90',
 'opp_team_goals_assists_pens_per90',
 'opp_team_xg_per90',
 'opp_team_gk_goals_against_per90',
 'opp_team_gk_save_pct',
 'opp_team_gk_clean_sheets_pct',
 'opp_team_passes_pct',
 'opp_team_passes_pct_medium',
 'opp_team_passes_pct_long',
 'opp_team_sca_per90',
 'opp_team_gca_per90',
 #'opp_team_dribble_tackles_pct',
 'opp_team_aerials_won_pct',
 'opp_vs_team_possession',
 'opp_vs_team_goals_per90',
 'opp_vs_team_assists_per90',
 'opp_vs_team_xg_per90',
 'opp_vs_team_gk_save_pct',
 'opp_vs_team_gk_clean_sheets_pct',
 'opp_vs_team_gk_pct_passes_launched',
 'opp_vs_team_gk_crosses_stopped_pct',
 'opp_vs_team_shots_on_target_per90',
 'opp_vs_team_passes_pct',
 'opp_vs_team_passes_pct_short',
 'opp_vs_team_passes_pct_medium',
 'opp_vs_team_passes_pct_long',
 'opp_vs_team_sca_per90',
 'opp_vs_team_gca_per90',
 #'opp_vs_team_dribble_tackles_pct',
 #'opp_vs_team_dribbles_completed_pct',
 'opp_vs_team_aerials_won_pct',
                
 'vote_avg',
 'vote_std']

features_rel = [
 'goals',
 'assists',
 'cards_yellow',
 'cards_red',
 'xg',
 'npxg',
 'shots_on_target',
 'passes_completed',
 'passes_into_final_third',
 'passes_into_penalty_area',
 'progressive_passes',
 'passes_live',
 'passes_dead',
 'through_balls',
 'passes_switches',
 'crosses',
 'corner_kicks',
 #'dribble_tackles',
 #'dribbles_vs',
 #'dribbled_past',
 'blocks',
 'blocked_shots',
 'blocked_passes',
 'interceptions',
 'clearances',
 'errors',
 'touches',
 'touches_def_pen_area',
 'touches_def_3rd',
 'touches_mid_3rd',
 'touches_att_3rd',
 'touches_att_pen_area',
 'touches_live_ball',
 #'dribbles_completed',
 #'dribbles',
 'passes_received',
 'miscontrols',
 'dispossessed',
 'fouls',
 'fouled',
 'aerials_won',
 'aerials_lost',
 'carries',
 'progressive_carries',
 'carries_into_final_third',
 'carries_into_penalty_area']

features_rel_gamecorr = [
    'goals',
    'assists',
    'xg',
    'npxg',
    'cards_yellow',
    'cards_red'
]

Function for merging match data (player, team, opponent team) and select only the desired features, applying the desired corrections.

In [ ]:
def player_match_data_ext(player, pteam, oppteam):
    pdata = player_match_data(player, pteam, oppteam)
    
    if(not isinstance(pdata, pd.DataFrame)):
        return None
    
    out = pd.concat([pdata[features_abs], pdata[features_rel]], axis = 1)
    
    out[features_rel] = out[features_rel] / max(pdata['minutes'][0], 1)
    
    out[features_rel_gamecorr] = out[features_rel_gamecorr] * (pdata['minutes'][0] / max(pdata['games'][0], 1) / 90)
    
    return out
    

Columns variable for the match data, to be then added to votes data.

In [ ]:
test = player_match_data_ext('Barella', 'Inter', 'Atalanta')

add_columns = test.columns

test

Load Fantacalcio votes data for each match of the season.

In [ ]:
votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)
votes

Merge votes data with match data for the player.

In [ ]:
db = votes.copy()

delta = pd.DataFrame(columns = add_columns)

for i in range(db.shape[0]):
    ext_data = player_match_data_ext(db['player'][i], db['team'][i], db['oppteam'][i])
    
    if(not isinstance(ext_data, pd.DataFrame)):
        continue
    
    ext_data = ext_data.rename(index = {db['player'][i] : i})
    
    delta = pd.concat([delta, ext_data], axis = 0)
    
    if(i % 100 == 0):
        print(i)
    
db = pd.concat([db, delta], axis = 1)
    

In [ ]:
db

Remove undesired database entries.

In [ ]:
db = db.drop(db[db.r == 'P'].index) # remove goalkeepers
db = db.drop(db[db.r != db.r].index) # remove non existing players (by searching NaN values)
db = db.drop(db[db.xg != db.xg].index)


In [ ]:
db

Save outfield players database to file.

In [ ]:
db.to_excel('mid_outputs/database_entries.xlsx')

Do the same process for goalkeepers, generating a different database.

In [ ]:
for i in range(db.columns.shape[0]):
    print(str(i) + " - " + str(db.columns[i]))

In [ ]:
rx = pd.read_excel('tmp/playermatchdata_sel_gk.xlsx', index_col = 0, header = None)   

out = list()

for i in range(1, rx.shape[1]):
        if(rx[i][2] == "x"):
            out.append(rx[i][0])
            
out

In [ ]:
features_abs_gk = [
 'gk_games',
 'gk_games_starts',
 'gk_minutes',
 'gk_goals_against_per90', 
 'gk_save_pct',
 'gk_clean_sheets_pct',
 'gk_psxg_net_per90',
 'gk_passes_pct_launched',
 'gk_pct_passes_launched',
 'gk_passes_length_avg',
 'gk_pct_goal_kicks_launched',
 'gk_goal_kick_length_avg',
 'gk_crosses_stopped_pct',
 'gk_def_actions_outside_pen_area_per90',
 'gk_avg_distance_def_actions',
    
 'team_possession',
 'team_goals_assists_per90',
 'team_goals_pens_per90',
 'team_goals_assists_pens_per90',
 'team_xg_per90',
 'team_gk_goals_against_per90',
 'team_gk_save_pct',
 'team_gk_clean_sheets_pct',
 'team_passes_pct',
 'team_passes_pct_medium',
 'team_passes_pct_long',
 'team_sca_per90',
 'team_gca_per90',
 #'team_dribble_tackles_pct',
 'team_aerials_won_pct',
 'vs_team_possession',
 'vs_team_goals_per90',
 'vs_team_assists_per90',
 'vs_team_xg_per90',
 'vs_team_gk_save_pct',
 'vs_team_gk_clean_sheets_pct',
 'vs_team_gk_pct_passes_launched',
 'vs_team_gk_crosses_stopped_pct',
 'vs_team_shots_on_target_per90',
 'vs_team_passes_pct',
 'vs_team_passes_pct_short',
 'vs_team_passes_pct_medium',
 'vs_team_passes_pct_long',
 'vs_team_sca_per90',
 'vs_team_gca_per90',
 #'vs_team_dribble_tackles_pct',
 #'vs_team_dribbles_completed_pct',
 'vs_team_aerials_won_pct',
 'opp_team_possession',
 'opp_team_goals_assists_per90',
 'opp_team_goals_pens_per90',
 'opp_team_goals_assists_pens_per90',
 'opp_team_xg_per90',
 'opp_team_gk_goals_against_per90',
 'opp_team_gk_save_pct',
 'opp_team_gk_clean_sheets_pct',
 'opp_team_passes_pct',
 'opp_team_passes_pct_medium',
 'opp_team_passes_pct_long',
 'opp_team_sca_per90',
 'opp_team_gca_per90',
 #'opp_team_dribble_tackles_pct',
 'opp_team_aerials_won_pct',
 'opp_vs_team_possession',
 'opp_vs_team_goals_per90',
 'opp_vs_team_assists_per90',
 'opp_vs_team_xg_per90',
 'opp_vs_team_gk_save_pct',
 'opp_vs_team_gk_clean_sheets_pct',
 'opp_vs_team_gk_pct_passes_launched',
 'opp_vs_team_gk_crosses_stopped_pct',
 'opp_vs_team_shots_on_target_per90',
 'opp_vs_team_passes_pct',
 'opp_vs_team_passes_pct_short',
 'opp_vs_team_passes_pct_medium',
 'opp_vs_team_passes_pct_long',
 'opp_vs_team_sca_per90',
 'opp_vs_team_gca_per90',
 #'opp_vs_team_dribble_tackles_pct',
 #'opp_vs_team_dribbles_completed_pct',
 'opp_vs_team_aerials_won_pct',
                
 'vote_avg',
 'vote_std']

features_rel_gk = [
 'gk_shots_on_target_against',
 'gk_saves',
 'gk_free_kick_goals_against',
 'gk_corner_kick_goals_against',
 'gk_own_goals_against',
 'gk_psxg',
 'gk_psnpxg_per_shot_on_target_against',
 'gk_psxg_net',
 'gk_passes_completed_launched',
 'gk_passes_launched',
 'gk_passes',
 'gk_passes_throws',
 'gk_goal_kicks',
 'gk_crosses',
 'gk_crosses_stopped',
]

In [ ]:
def player_match_data_ext_gk(player, pteam, oppteam):
    pdata = player_match_data(player, pteam, oppteam)
    
    if(not isinstance(pdata, pd.DataFrame)):
        return None
    
    if(pdata['gk_games'][0] <= 0):
        return None
    
    out = pd.concat([pdata[features_abs_gk], pdata[features_rel_gk]], axis = 1)
    
    out[features_rel_gk] = out[features_rel_gk] / max(pdata['minutes'][0], 1)

    return out
    

In [ ]:
test = player_match_data_ext_gk('Consigli', 'Sassuolo', 'Atalanta')

add_columns_gk = test.columns

test

In [ ]:
db_gk = votes.copy()

delta = pd.DataFrame(columns = add_columns_gk)

for i in range(db_gk.shape[0]):
    ext_data = player_match_data_ext_gk(db_gk['player'][i], db_gk['team'][i], db_gk['oppteam'][i])
    
    if(i % 100 == 0):
        print(i)
        
    if(not isinstance(ext_data, pd.DataFrame)):
        continue
    
    ext_data = ext_data.rename(index = {db_gk['player'][i] : i})
    
    delta = pd.concat([delta, ext_data], axis = 0)
    
    
    
db_gk = pd.concat([db_gk, delta], axis = 1)
    

In [ ]:
db_gk

In [ ]:
db_gk = db_gk.drop(db_gk[db_gk.gk_games != db_gk.gk_games].index) # remove non extisting players

In [ ]:
db_gk

Save goalkeepers players database to file.

In [ ]:
db_gk.to_excel('mid_outputs/database_entries_gk.xlsx')

In [ ]:
for i in range(db_gk.columns.shape[0]):
    print(str(i) + " - " + str(db_gk.columns[i]))